In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

train_dir = '../input/herbarium-2022-fgvc9/train_images/'
test_dir = '../input/herbarium-2022-fgvc9/test_images/'

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

In [2]:
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [3]:
"""!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           
import gc
gc.collect()
torch.cuda.empty_cache()"""

'!pip install GPUtil\n\nimport torch\nfrom GPUtil import showUtilization as gpu_usage\nfrom numba import cuda\n\ndef free_gpu_cache():\n    print("Initial GPU Usage")\n    gpu_usage()                             \n\n    torch.cuda.empty_cache()\n\n    cuda.select_device(0)\n    cuda.close()\n    cuda.select_device(0)\n\n    print("GPU Usage after emptying the cache")\n    gpu_usage()\n\nfree_gpu_cache()                           \nimport gc\ngc.collect()\ntorch.cuda.empty_cache()'

In [4]:
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

test_ids = [image['image_id'] for image in test_meta]
test_dirs = [test_dir + image['file_name'] for image in test_meta]

#Create the initial training dataframe with the above defined columns
main_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

#Create a testing dataframe
Never_used = pd.DataFrame({
    "test_id" : test_ids,
    "test_dir" : test_dirs
})

main_df

,image_id,image_dir,category,genus
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,1
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,1
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,1
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,1
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,1
...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,2584


In [5]:
#Add a genus column to the dataframe
genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
main_df['genus'] = main_df['genus'].map(genus_map)

##Create a family column in the datagframe based on the genus names
    # Step 1: Create dictionary of genus -> family mapping
genus_family_map = {}
for category in train_meta["categories"]:
    genus = category['genus']
    family = category['family']
    genus_family_map[genus] = family

    # Step 2: Create new column with default value of None™
main_df['family'] = None

    # Step 3: Update values in new column based on genus -> family mapping
for i, row in main_df.iterrows():
    genus = row['genus']
    if genus in genus_family_map:
        family = genus_family_map[genus]
        main_df.at[i, 'family'] = family

main_df

,image_id,image_dir,category,genus,family
0,00000__001,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
1,00000__002,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
2,00000__003,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
3,00000__004,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
4,00000__005,../input/herbarium-2022-fgvc9/train_images/000...,0,Abies,Pinaceae
...,...,...,...,...,...
839767,15504__032,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839768,15504__033,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839769,15504__035,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae
839770,15504__036,../input/herbarium-2022-fgvc9/train_images/155...,15504,Zygophyllum,Zygophyllaceae


In [6]:
#Filter only the images of plants that are in the Poaceae family
main_df = main_df.loc[main_df['family'] == 'Poaceae']
#Reset index
main_df = main_df.reset_index(drop=True)

main_df

,image_id,image_dir,category,genus,family
0,00333__001,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
1,00333__002,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
2,00333__003,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
3,00333__004,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
4,00333__005,../input/herbarium-2022-fgvc9/train_images/003...,333,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53543,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53544,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae
53545,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,15501,Zuloagaea,Poaceae


In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from PIL import Image
import timm

!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=cba873fbed14aac36cbc07d3b3fa951a51b2bb1d534f5df0dc8fcbd9c2ed4a13
  Stored in directory: /root/.cache/pip/wheels/12/a2/36/21b9bde5f8deeeb6312efe88ddde26a51facbd2089f32917b3
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=c431f0d195af0eb5691eed7b49acc532ed7d16f4b969a50ca533dbb7804ae0da
  Stored in directory: /root/.cache/pip/wheels/96/9a/78/61eeeec98da40f44085da9ba3fec952b4ab7224f5c5be75126
Successfully built fvcore iopath


In [8]:
main_df['category'] = pd.factorize(main_df['category'])[0]
main_df

,image_id,image_dir,category,genus,family
0,00333__001,../input/herbarium-2022-fgvc9/train_images/003...,0,Agrostis,Poaceae
1,00333__002,../input/herbarium-2022-fgvc9/train_images/003...,0,Agrostis,Poaceae
2,00333__003,../input/herbarium-2022-fgvc9/train_images/003...,0,Agrostis,Poaceae
3,00333__004,../input/herbarium-2022-fgvc9/train_images/003...,0,Agrostis,Poaceae
4,00333__005,../input/herbarium-2022-fgvc9/train_images/003...,0,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,../input/herbarium-2022-fgvc9/train_images/155...,963,Zuloagaea,Poaceae
53543,15501__103,../input/herbarium-2022-fgvc9/train_images/155...,963,Zuloagaea,Poaceae
53544,15501__105,../input/herbarium-2022-fgvc9/train_images/155...,963,Zuloagaea,Poaceae
53545,15501__106,../input/herbarium-2022-fgvc9/train_images/155...,963,Zuloagaea,Poaceae


In [9]:
grouped = main_df.groupby('category')
# Split the dataset for each class separately
train_dfs = []
test_dfs = []

for name, group in grouped:
    # Split the group into train and test sets
    train, test = train_test_split(group, test_size=0.2, random_state=42)
    # Add the train and test dataframes to the respective lists
    train_dfs.append(train)
    test_dfs.append(test)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df_initial = pd.concat(train_dfs)
train_df_initial = train_df_initial.reset_index(drop=True)

test_df = pd.concat(test_dfs)
test_df = test_df.reset_index(drop=True)

In [10]:
grouped2 = train_df_initial.groupby('category')
# Split the dataset for each class separately
train_dfs = []
val_dfs = []

for name, group in grouped2:
    # Split the group into train and test sets
    train, validation = train_test_split(group, test_size=0.2, random_state=42)
    # Add the train and test dataframes to the respective lists
    train_dfs.append(train)
    val_dfs.append(validation)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df = pd.concat(train_dfs)
train_df = train_df.reset_index(drop=True)

val_df = pd.concat(val_dfs)
val_df = val_df.reset_index(drop=True)

In [11]:
batch_size = 32
epochs = 8
IM_SIZE = 256
learning_rate = 1e-4

X_train, Y_train = train_df["image_dir"].values, train_df["category"].values

X_val, Y_val = val_df["image_dir"].values, val_df["category"].values

X_test, Y_test  = test_df["image_dir"].values, test_df["category"].values

Transform = transforms.Compose([
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


In [12]:
class GetData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
    
        if "train" in self.fnames[index]:             
            return self.transform(x), self.labels[index]
        elif "test" in self.fnames[index]:            
            return self.transform(x), self.fnames[index]
        
        
class GetTestData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
        return self.transform(x), self.labels[index]

        
trainset = GetData(X_train, Y_train, Transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

valset = GetData(X_val, Y_val, Transform)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)

testset = GetTestData(X_test, Y_test, Transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest101', pretrained=True)            

/opt/conda/lib/python3.7/site-packages/torch/hub.py:268: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/zhanghang1989/ResNeSt/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/zhanghang1989/ResNeSt/releases/download/weights_step1/resnest101-22405ba7.pth" to /root/.cache/torch/hub/checkpoints/resnest1

  0%|          | 0.00/185M [00:00<?, ?B/s]

In [13]:
num_classes = train_df['category'].nunique()
print(num_classes)

964


In [14]:
total_layers = len(list(model.parameters()))

for param in model.parameters():
    param.requires_grad = True
    
n_inputs = model.fc.in_features
last_layer = nn.Linear(n_inputs, num_classes)
model.fc = last_layer 
if torch.cuda.is_available():
    model.cuda()
print(model.fc.out_features)    

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

964


In [15]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def train(trainloader, model, criterion, optimizer, scaler, device=torch.device("cpu")):
    train_acc = 0.0
    train_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(images)
            loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            acc = ((output.argmax(dim=1) == labels).float().mean())
            train_acc += acc
            train_loss += loss
            y_true += labels.cpu().numpy().tolist()
            y_pred += output.argmax(dim=1).cpu().numpy().tolist()
            
    train_f1 = f1_score(y_true, y_pred, average=None)
    train_f1_avg = f1_score(y_true, y_pred, average='macro')
    return train_acc/len(trainloader), train_loss/len(trainloader), train_f1, train_f1_avg

In [16]:
def evaluate(testloader, model, criterion, device=torch.device("cpu")):
    model.eval()
    eval_acc = 0.0
    eval_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(testloader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
        acc = ((output.argmax(dim=1) == labels).float().mean())
        eval_acc += acc
        eval_loss += loss
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()
  
    eval_f1 = f1_score(y_true, y_pred, average=None)
    eval_f1_avg = f1_score(y_true, y_pred, average='macro')
    return eval_acc/len(testloader), eval_loss/len(testloader), eval_f1, eval_f1_avg

In [17]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores


for epoch in range(epochs):
    train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

    train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
    val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch

    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")
    
    print("F1 score per class (Train):")
    for i, f1 in enumerate(train_f1):
        print(f"Class {i}: {f1}")

    print("\nF1 score per class (Validation):") 
    for i, f1 in enumerate(eval_f1):
        print(f"Class {i}: {f1}")

    print("===="*8)

100%|██████████| 278/278 [04:31<00:00,  1.03it/s]


Epoch 1 | Train Acc: 17.772958755493164 | Train Loss: 4.574265480041504 | Train F1 (Avg): 0.14608366344883164
	 Val Acc: 40.38519287109375 | Val Loss: 2.5865890979766846 | Val F1 (Avg): 0.32221381595960086
F1 score per class (Train):
Class 0: 0.0
Class 1: 0.0
Class 2: 0.0
Class 3: 0.06060606060606061
Class 4: 0.23376623376623376
Class 5: 0.19999999999999998
Class 6: 0.04761904761904762
Class 7: 0.0
Class 8: 0.24390243902439024
Class 9: 0.24175824175824176
Class 10: 0.0
Class 11: 0.0
Class 12: 0.09523809523809525
Class 13: 0.07017543859649122
Class 14: 0.3170731707317073
Class 15: 0.0
Class 16: 0.1702127659574468
Class 17: 0.2522522522522523
Class 18: 0.0
Class 19: 0.0
Class 20: 0.09523809523809525
Class 21: 0.22641509433962265
Class 22: 0.06557377049180328
Class 23: 0.125
Class 24: 0.2173913043478261
Class 25: 0.0
Class 26: 0.0
Class 27: 0.05194805194805195
Class 28: 0.29411764705882354
Class 29: 0.0
Class 30: 0.18947368421052632
Class 31: 0.13461538461538464
Class 32: 0.06666666666666

100%|██████████| 278/278 [03:50<00:00,  1.21it/s]


Epoch 2 | Train Acc: 47.38605499267578 | Train Loss: 2.1361806392669678 | Train F1 (Avg): 0.41907713681843783
	 Val Acc: 51.19029998779297 | Val Loss: 2.0115065574645996 | Val F1 (Avg): 0.45867279165533686
F1 score per class (Train):
Class 0: 0.0
Class 1: 0.18181818181818182
Class 2: 0.39999999999999997
Class 3: 0.4375
Class 4: 0.4615384615384615
Class 5: 0.3448275862068966
Class 6: 0.17857142857142855
Class 7: 0.6666666666666666
Class 8: 0.39999999999999997
Class 9: 0.46601941747572817
Class 10: 0.27906976744186046
Class 11: 0.2222222222222222
Class 12: 0.3170731707317073
Class 13: 0.2891566265060241
Class 14: 0.5346534653465346
Class 15: 0.24390243902439027
Class 16: 0.33333333333333326
Class 17: 0.5384615384615384
Class 18: 0.1379310344827586
Class 19: 0.16666666666666666
Class 20: 0.42857142857142855
Class 21: 0.5263157894736842
Class 22: 0.3695652173913044
Class 23: 0.5333333333333333
Class 24: 0.6170212765957447
Class 25: 0.4
Class 26: 0.0
Class 27: 0.6732673267326733
Class 28: 0

100%|██████████| 278/278 [04:00<00:00,  1.16it/s]


...

Epoch 6 | Train Acc: 86.2155532836914 | Train Loss: 0.4448815584182739 | Train F1 (Avg): 0.8605050599999626
	 Val Acc: 60.21765899658203 | Val Loss: 1.851114273071289 | Val F1 (Avg): 0.5707972372310349
F1 score per class (Train):
Class 0: 0.888888888888889
Class 1: 0.7692307692307693
Class 2: 0.9333333333333333
Class 3: 0.8421052631578947
Class 4: 0.9361702127659575
Class 5: 0.8135593220338982
Class 6: 0.7169811320754716
Class 7: 1.0
Class 8: 0.7826086956521738
Class 9: 0.7
Class 10: 0.7826086956521738
Class 11: 0.888888888888889
Class 12: 0.6136363636363638
Class 13: 0.717391304347826
Class 14: 0.7956989247311829
Class 15: 0.723404255319149
Class 16: 0.7857142857142856
Class 17: 0.7640449438202247
Class 18: 0.689655172413793
Class 19: 0.9333333333333333
Class 20: 0.8799999999999999
Class 21: 0.8387096774193549
Class 22: 0.7555555555555555
Class 23: 1.0
Class 24: 0.891304347826087
Class 25: 1.0
Class 26: 0.7272727272727272
Class 27: 0.968421052631579
Class 28: 0.9199999999999999
Class 

100%|██████████| 278/278 [03:27<00:00,  1.34it/s]


Epoch 7 | Train Acc: 89.34864044189453 | Train Loss: 0.3309480547904968 | Train F1 (Avg): 0.8934917681586312
	 Val Acc: 61.641109466552734 | Val Loss: 1.956449031829834 | Val F1 (Avg): 0.5863216337009103
F1 score per class (Train):
Class 0: 1.0
Class 1: 0.8571428571428571
Class 2: 0.9285714285714286
Class 3: 0.8823529411764706
Class 4: 0.9052631578947369
Class 5: 0.8307692307692308
Class 6: 0.7692307692307693
Class 7: 1.0
Class 8: 0.8275862068965518
Class 9: 0.8085106382978724
Class 10: 0.8333333333333334
Class 11: 0.8444444444444444
Class 12: 0.7777777777777778
Class 13: 0.7526881720430108
Class 14: 0.851063829787234
Class 15: 0.8076923076923077
Class 16: 0.8666666666666667
Class 17: 0.8387096774193549
Class 18: 0.9655172413793104
Class 19: 0.9333333333333333
Class 20: 0.8
Class 21: 0.8631578947368421
Class 22: 0.8863636363636365
Class 23: 0.8666666666666667
Class 24: 0.9318181818181819
Class 25: 1.0
Class 26: 1.0
Class 27: 0.9583333333333334
Class 28: 0.9484536082474228
Class 29: 0.8

100%|██████████| 278/278 [03:31<00:00,  1.32it/s]

Epoch 8 | Train Acc: 91.78868865966797 | Train Loss: 0.26178058981895447 | Train F1 (Avg): 0.9189115403961944
	 Val Acc: 61.38256072998047 | Val Loss: 1.9266481399536133 | Val F1 (Avg): 0.5773473514597228
F1 score per class (Train):
Class 0: 1.0
Class 1: 0.9333333333333333
Class 2: 1.0
Class 3: 0.9473684210526315
Class 4: 0.9473684210526315
Class 5: 0.96875
Class 6: 0.8235294117647058
Class 7: 1.0
Class 8: 0.7954545454545455
Class 9: 0.8484848484848485
Class 10: 0.6956521739130435
Class 11: 0.8837209302325582
Class 12: 0.782608695652174
Class 13: 0.8421052631578947
Class 14: 0.9090909090909091
Class 15: 0.8444444444444444
Class 16: 0.819672131147541
Class 17: 0.7692307692307693
Class 18: 0.8148148148148148
Class 19: 1.0
Class 20: 0.810126582278481
Class 21: 0.8842105263157894
Class 22: 0.8444444444444444
Class 23: 0.9333333333333333
Class 24: 0.9347826086956522
Class 25: 1.0
Class 26: 0.888888888888889
Class 27: 0.9894736842105264
Class 28: 0.9591836734693877
Class 29: 0.82758620689655

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train_f1_scores, label="Train F1 (Avg)")
plt.plot(val_f1_scores, label="Validation F1 (Avg)")
plt.xlabel("Epoch")
plt.ylabel("F1 Score")
plt.title(f"F1 Scores Over Epochs")
plt.legend()
plt.xticks(range(len(train_f1_scores)), range(1, len(train_f1_scores) + 1))
plt.show()

In [20]:
from sklearn.metrics import classification_report, f1_score

# Assume that the `testloader` DataLoader has been created for the test set
test_acc = 0.0
y_true = []
y_pred = []

model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Loop through the test set
with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        output = model(images)

        # Calculate accuracy
        acc = (output.argmax(dim=1) == labels).float().mean()
        test_acc += acc

        # Store true labels and predicted labels for other metrics
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()

# Calculate the final test accuracy
test_acc = test_acc / len(testloader)
print(f"Test accuracy: {test_acc:.2%}")

# Calculate the classification report, which includes the F1 score for all classes
report = classification_report(y_true, y_pred, output_dict=True)

# Print the classification report
for class_label, metrics in report.items():
    if class_label.isdigit():
        print(f"Class {class_label}: F1 score = {metrics['f1-score']:.2f}")

Testing: 100%|██████████| 347/347 [05:25<00:00,  1.07it/s]

Test accuracy: 62.43%
Class 0: F1 score = 0.50
Class 1: F1 score = 0.00
Class 2: F1 score = 0.00
Class 3: F1 score = 0.50
Class 4: F1 score = 0.75
Class 5: F1 score = 0.48
Class 6: F1 score = 0.19
Class 7: F1 score = 0.67
Class 8: F1 score = 0.62
Class 9: F1 score = 0.33
Class 10: F1 score = 0.53
Class 11: F1 score = 0.55
Class 12: F1 score = 0.38
Class 13: F1 score = 0.36
Class 14: F1 score = 0.48
Class 15: F1 score = 0.57
Class 16: F1 score = 0.33
Class 17: F1 score = 0.62
Class 18: F1 score = 0.50
Class 19: F1 score = 0.80
Class 20: F1 score = 0.19
Class 21: F1 score = 0.56
Class 22: F1 score = 0.67
Class 23: F1 score = 0.80
Class 24: F1 score = 0.83
Class 25: F1 score = 0.00
Class 26: F1 score = 0.00
Class 27: F1 score = 0.81
Class 28: F1 score = 0.85
Class 29: F1 score = 0.32
Class 30: F1 score = 0.93
Class 31: F1 score = 0.80
Class 32: F1 score = 0.58
Class 33: F1 score = 0.62
Class 34: F1 score = 0.58
Class 35: F1 score = 0.80
Class 36: F1 score = 0.64
Class 37: F1 score = 0.69



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
avg_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Average F1 score: {avg_f1:.2f}")

Average F1 score: 0.59


In [22]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores


train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch

print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")

print("F1 score per class (Train):")
for i, f1 in enumerate(train_f1):
    print(f"Class {i}: {f1}")

print("\nF1 score per class (Validation):") 
for i, f1 in enumerate(eval_f1):
    print(f"Class {i}: {f1}")

print("===="*8)

100%|██████████| 278/278 [03:36<00:00,  1.28it/s]

Epoch 8 | Train Acc: 93.3197250366211 | Train Loss: 0.20771105587482452 | Train F1 (Avg): 0.9330289083601799
	 Val Acc: 61.25474548339844 | Val Loss: 2.0824573040008545 | Val F1 (Avg): 0.5794574804596482
F1 score per class (Train):
Class 0: 1.0
Class 1: 0.9333333333333333
Class 2: 0.8571428571428571
Class 3: 0.8500000000000001
Class 4: 0.9462365591397849
Class 5: 0.7936507936507936
Class 6: 0.8679245283018868
Class 7: 1.0
Class 8: 0.8604651162790699
Class 9: 0.9183673469387755
Class 10: 0.8979591836734694
Class 11: 0.9545454545454546
Class 12: 0.851063829787234
Class 13: 0.9375
Class 14: 0.9484536082474228
Class 15: 0.8571428571428572
Class 16: 0.8852459016393444
Class 17: 0.9130434782608695
Class 18: 0.8
Class 19: 0.9411764705882353
Class 20: 0.8533333333333333
Class 21: 0.9583333333333333
Class 22: 0.9176470588235294
Class 23: 0.967741935483871
Class 24: 0.9555555555555556
Class 25: 1.0
Class 26: 0.888888888888889
Class 27: 1.0
Class 28: 1.0
Class 29: 0.896551724137931
Class 30: 0.98

In [23]:
from sklearn.metrics import classification_report, f1_score

# Assume that the `testloader` DataLoader has been created for the test set
test_acc = 0.0
y_true = []
y_pred = []

model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Loop through the test set
with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        output = model(images)

        # Calculate accuracy
        acc = (output.argmax(dim=1) == labels).float().mean()
        test_acc += acc

        # Store true labels and predicted labels for other metrics
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()

# Calculate the final test accuracy
test_acc = test_acc / len(testloader)
print(f"Test accuracy: {test_acc:.2%}")

# Calculate the classification report, which includes the F1 score for all classes
report = classification_report(y_true, y_pred, output_dict=True)

# Print the classification report
for class_label, metrics in report.items():
    if class_label.isdigit():
        print(f"Class {class_label}: F1 score = {metrics['f1-score']:.2f}")

Testing: 100%|██████████| 347/347 [04:59<00:00,  1.16it/s]

Test accuracy: 63.36%
Class 0: F1 score = 0.50
Class 1: F1 score = 0.00
Class 2: F1 score = 0.50
Class 3: F1 score = 0.00
Class 4: F1 score = 0.74
Class 5: F1 score = 0.34
Class 6: F1 score = 0.22
Class 7: F1 score = 0.67
Class 8: F1 score = 0.22
Class 9: F1 score = 0.46
Class 10: F1 score = 0.31
Class 11: F1 score = 0.38
Class 12: F1 score = 0.30
Class 13: F1 score = 0.40
Class 14: F1 score = 0.40
Class 15: F1 score = 0.31
Class 16: F1 score = 0.43
Class 17: F1 score = 0.39
Class 18: F1 score = 0.22
Class 19: F1 score = 0.80
Class 20: F1 score = 0.44
Class 21: F1 score = 0.52
Class 22: F1 score = 0.43
Class 23: F1 score = 0.55
Class 24: F1 score = 0.93
Class 25: F1 score = 0.00
Class 26: F1 score = 0.40
Class 27: F1 score = 0.84
Class 28: F1 score = 0.85
Class 29: F1 score = 0.50
Class 30: F1 score = 0.82
Class 31: F1 score = 0.89
Class 32: F1 score = 0.62
Class 33: F1 score = 0.67
Class 34: F1 score = 0.83
Class 35: F1 score = 0.79
Class 36: F1 score = 0.71
Class 37: F1 score = 0.71



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
avg_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Average F1 score: {avg_f1:.2f}")

Average F1 score: 0.60


In [25]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores


train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch

print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")

print("F1 score per class (Train):")
for i, f1 in enumerate(train_f1):
    print(f"Class {i}: {f1}")

print("\nF1 score per class (Validation):") 
for i, f1 in enumerate(eval_f1):
    print(f"Class {i}: {f1}")

print("===="*8)

100%|██████████| 278/278 [04:33<00:00,  1.02it/s]

Epoch 8 | Train Acc: 94.39285278320312 | Train Loss: 0.17521609365940094 | Train F1 (Avg): 0.943188342882832
	 Val Acc: 61.18855285644531 | Val Loss: 2.1170711517333984 | Val F1 (Avg): 0.5804411256142491
F1 score per class (Train):
Class 0: 1.0
Class 1: 0.9411764705882353
Class 2: 1.0
Class 3: 0.918918918918919
Class 4: 0.9791666666666666
Class 5: 0.9354838709677419
Class 6: 0.9056603773584906
Class 7: 1.0
Class 8: 0.9302325581395349
Class 9: 0.9375
Class 10: 0.9583333333333334
Class 11: 0.9767441860465117
Class 12: 0.8222222222222223
Class 13: 0.875
Class 14: 0.9375
Class 15: 0.9166666666666666
Class 16: 0.9655172413793104
Class 17: 0.9032258064516129
Class 18: 0.9285714285714286
Class 19: 1.0
Class 20: 0.9113924050632911
Class 21: 0.9032258064516129
Class 22: 0.8571428571428571
Class 23: 1.0
Class 24: 0.9302325581395349
Class 25: 1.0
Class 26: 1.0
Class 27: 0.9473684210526315
Class 28: 0.9690721649484536
Class 29: 0.8666666666666667
Class 30: 0.9361702127659574
Class 31: 0.9772727272

In [26]:
from sklearn.metrics import classification_report, f1_score

# Assume that the `testloader` DataLoader has been created for the test set
test_acc = 0.0
y_true = []
y_pred = []

model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Loop through the test set
with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        output = model(images)

        # Calculate accuracy
        acc = (output.argmax(dim=1) == labels).float().mean()
        test_acc += acc

        # Store true labels and predicted labels for other metrics
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()

# Calculate the final test accuracy
test_acc = test_acc / len(testloader)
print(f"Test accuracy: {test_acc:.2%}")

# Calculate the classification report, which includes the F1 score for all classes
report = classification_report(y_true, y_pred, output_dict=True)

# Print the classification report
for class_label, metrics in report.items():
    if class_label.isdigit():
        print(f"Class {class_label}: F1 score = {metrics['f1-score']:.2f}")

Testing: 100%|██████████| 347/347 [04:21<00:00,  1.33it/s]

Test accuracy: 63.54%
Class 0: F1 score = 0.50
Class 1: F1 score = 0.00
Class 2: F1 score = 0.50
Class 3: F1 score = 0.00
Class 4: F1 score = 0.81
Class 5: F1 score = 0.44
Class 6: F1 score = 0.32
Class 7: F1 score = 1.00
Class 8: F1 score = 0.46
Class 9: F1 score = 0.44
Class 10: F1 score = 0.59
Class 11: F1 score = 0.44
Class 12: F1 score = 0.54
Class 13: F1 score = 0.40
Class 14: F1 score = 0.69
Class 15: F1 score = 0.25
Class 16: F1 score = 0.37
Class 17: F1 score = 0.55
Class 18: F1 score = 0.33
Class 19: F1 score = 0.67
Class 20: F1 score = 0.48
Class 21: F1 score = 0.65
Class 22: F1 score = 0.50
Class 23: F1 score = 0.67
Class 24: F1 score = 0.79
Class 25: F1 score = 0.00
Class 26: F1 score = 0.67
Class 27: F1 score = 0.82
Class 28: F1 score = 0.89
Class 29: F1 score = 0.29
Class 30: F1 score = 0.87
Class 31: F1 score = 0.93
Class 32: F1 score = 0.65
Class 33: F1 score = 0.69
Class 34: F1 score = 0.75
Class 35: F1 score = 0.69
Class 36: F1 score = 0.79
Class 37: F1 score = 0.62



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
avg_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Average F1 score: {avg_f1:.2f}")

Average F1 score: 0.60
